In [25]:
!pip install streamlit pyngrok

In [26]:
!pip install llama-cpp-python

In [ ]:
from huggingface_hub import login
login("YOUR_HUGGINGFACE_TOKEN")

In [40]:
%%writefile app.py

import streamlit as st
import torch
from diffusers import StableDiffusionXLPipeline
from transformers import pipeline
import os
import zipfile
from io import BytesIO

# Set Page Configuration
st.set_page_config(page_title="AI Comic Crafter", page_icon="🦸", layout="wide")

# Load Stable Diffusion Model (Hugging Face Diffusers)
@st.cache_resource
def load_sd_model():
    model_id = "stabilityai/stable-diffusion-xl-base-1.0"
    pipe = StableDiffusionXLPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
    )
    pipe.to("cuda" if torch.cuda.is_available() else "cpu")
    return pipe

pipe = load_sd_model()

# Load a Smaller LLM Model (GPT-2 for Text Generation)
@st.cache_resource
def load_llm():
    return pipeline("text-generation", model="gpt2")

llm = load_llm()

# Custom Styling for UI
st.markdown(
    """
    <style>
        .title-text {
            text-align: center;
            font-size: 42px;
            font-weight: bold;
            color: #ff4b4b;
        }
        .story-section {
            padding: 10px;
            background-color: #f5f5f5;
            border-radius: 10px;
        }
        .stButton>button {
            width: 100%;
            background-color: #ff4b4b;
            color: white !important;
            transition: background-color 0.3s ease-in-out;
        }
        .stButton>button:hover {
            background-color: #e03e3e;
        }
    </style>
    """, unsafe_allow_html=True
)

# Title
st.markdown('<div class="title-text">🎨 AI Comic Crafter 🦸</div>', unsafe_allow_html=True)
st.write("Create AI-powered comics with engaging stories and artistic visuals!")

# Sidebar Settings
st.sidebar.header("📜 **Story Settings**")
genre = st.sidebar.selectbox("Choose a Genre", ["Superhero", "Fantasy", "Sci-Fi", "Mystery"])
style = st.sidebar.selectbox("Comic Style", ["Manga", "Cartoon", "Realistic", "Vintage"])

# Generate Story Using LLM
if st.button("🤖 Generate AI Story"):
    with st.spinner("Generating story elements..."):
        prompt = f"Write a {genre} story. Introduction:"
        introduction = llm(prompt, max_length=50)[0]['generated_text']

        prompt = f"Continue the {genre} story with an exciting storyline:"
        storyline = llm(prompt, max_length=100)[0]['generated_text']

        prompt = f"Write the climax of the {genre} story:"
        climax = llm(prompt, max_length=50)[0]['generated_text']

        prompt = f"Conclude the {genre} story with a moral:"
        moral = llm(prompt, max_length=40)[0]['generated_text']

    # Update the existing text areas with generated content
    st.session_state["introduction"] = introduction
    st.session_state["storyline"] = storyline
    st.session_state["climax"] = climax
    st.session_state["moral"] = moral

# Main Story Input Section with Existing Fields
st.markdown("### 📝 **Enter Your Story Elements:**")
col1, col2 = st.columns(2)

with col1:
    introduction = st.text_area("🔹 Introduction", st.session_state.get("introduction", ""))
    climax = st.text_area("🔹 Climax", st.session_state.get("climax", ""))

with col2:
    storyline = st.text_area("🔹 Storyline", st.session_state.get("storyline", ""))
    moral = st.text_area("🔹 Moral", st.session_state.get("moral", ""))

# Function to Generate Comic-Style Images
def generate_comic_images(prompts):
    image_paths = []
    for i, prompt in enumerate(prompts):
        panel_prompt = f"{prompt}, {style} comic-style, highly detailed, vibrant colors"
        image = pipe(prompt=panel_prompt).images[0]  # Generate image
        image_path = f"comic_panel_{i+1}.png"
        image.save(image_path)
        image_paths.append(image_path)
    return image_paths

# Generate Story & Images
# Generate Story & Images
if st.button("🦸 Generate Comic Panels"):
    st.markdown("## 📖 **Generated Comic Panels:**")

    with st.spinner("Generating Comic Panels... 🖌️"):
        image_paths = generate_comic_images([introduction, storyline, climax, moral])

    story_parts = ["Introduction", "Storyline", "Climax", "Moral"]
    story_texts = [introduction, storyline, climax, moral]

    for i, img_path in enumerate(image_paths):
        # Display the story text before each image
        st.markdown(f"### {story_parts[i]}: {story_texts[i]}")
        st.image(img_path, use_container_width=True)

    # Zip and provide download button
    if image_paths:
        zip_buffer = BytesIO()
        with zipfile.ZipFile(zip_buffer, "w") as zip_file:
            for img_path in image_paths:
                zip_file.write(img_path)
        zip_buffer.seek(0)
        st.sidebar.download_button(
            label="📥 Download All Panels",
            data=zip_buffer,
            file_name="comic_panels.zip",
            mime="application/zip"
        )


# Footer
st.sidebar.markdown("---")
st.sidebar.write("✨ **Created by Arabinda Bhowmick** 🚀")


Overwriting app.py


In [41]:
!pkill -f ngrok

In [ ]:
!ngrok authtoken YOUR_NGROK_AUTHTOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [43]:
from pyngrok import ngrok
import os

# Run Streamlit in the background
os.system("streamlit run app.py &")

# Expose port 8501 (Streamlit's default port)
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at: {public_url}")

Streamlit app is live at: NgrokTunnel: "https://e22c-34-83-49-51.ngrok-free.app" -> "http://localhost:8501"
